In [4]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [2]:
from groq import Groq

client = Groq(
    api_key="",
)

In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    groq_api_key="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",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

response = llm.invoke("The first person to land an moon was...")
print(response.content)

Neil Armstrong.  He stepped onto the lunar surface on July 20, 1969, during the Apollo 11 mission.


In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.c-and-a.com/de/de/corporate/company/careers/jobs/data-science-intern-mfd-58038-en-us")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.









Data Science Intern (m/f/d) - Düsseldorf (DE) - C&A


   open menu  Go to C&A corporate homepage  Über uns Unternehmenskultur Nachhaltigkeit Highlight Projects News Karriere Kontakt   C&A Online-Shop C&A Filialfinder  Deutsch   Data Science Intern (m/f/d) Düsseldorf (DE) Apply Now The future looks like youThe Data Science Team in our Düsseldorf Headquarter is looking for a full-time Intern (m/f/d) for at least six months to support the team with their day-today business. You will get valuable insights in the data of an internationally operating company. As a Data Science / AI Intern, you will assist in a variety of projects that utilize data analytics, machine learning techniques, and cuttingedge AI technologies. Your role will involve supporting the analysis of complex data sets, aiding in the development of predictive models, and helping implement AI-driven solutions.Your responsibilities· Assist in the analysis and interpretation of data to derive actionable insights· Contrib

In [11]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ###INSTRUCTION:
    The Scraped text ist from the career's page of a website.
    your Job is to extract the Job postings and return them in JSON format containing the 
    following keys: "role", "experience", "skills" and "descriptions".
    Only return the valid JSON
    ### VALID JSON (No PREAMBLE):
    
    """
    
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke (input={"page_data":page_data})
type(res.content)


str

In [12]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Data Science Intern (m/f/d)',
  'experience': 'Intern for at least six months',
  'skills': ['ML',
   'Deep learning',
   'NLP',
   'Generative AI algorithms',
   'Python',
   'SQL',
   'Statistics'],
  'descriptions': ['Assist in the analysis and interpretation of data to derive actionable insights',
   'Contribute to the feature engineering process that improve the accuracy and performance of our models',
   'Support the development and application of predictive and forecasting models',
   'Contribute to projects involving natural language processing',
   'Research the application of the newest AI technologies']}]

In [13]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [14]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [18]:
links = collection.query(query_texts=["Experience in Python", "Expertise in React"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [19]:
json_res

[{'role': 'Data Science Intern (m/f/d)',
  'experience': 'Intern for at least six months',
  'skills': ['ML',
   'Deep learning',
   'NLP',
   'Generative AI algorithms',
   'Python',
   'SQL',
   'Statistics'],
  'descriptions': ['Assist in the analysis and interpretation of data to derive actionable insights',
   'Contribute to the feature engineering process that improve the accuracy and performance of our models',
   'Support the development and application of predictive and forecasting models',
   'Contribute to projects involving natural language processing',
   'Research the application of the newest AI technologies']}]

In [23]:
job

[{'role': 'Data Science Intern (m/f/d)',
  'experience': 'Intern for at least six months',
  'skills': ['ML',
   'Deep learning',
   'NLP',
   'Generative AI algorithms',
   'Python',
   'SQL',
   'Statistics'],
  'descriptions': ['Assist in the analysis and interpretation of data to derive actionable insights',
   'Contribute to the feature engineering process that improve the accuracy and performance of our models',
   'Support the development and application of predictive and forecasting models',
   'Contribute to projects involving natural language processing',
   'Research the application of the newest AI technologies']}]

In [24]:
job = json_res
skills = job[0]['skills']  # Accéder au premier élément de la liste puis à 'skills'
print(skills)


['ML', 'Deep learning', 'NLP', 'Generative AI algorithms', 'Python', 'SQL', 'Statistics']


In [25]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Science Solutions for Your Business Needs

Dear Hiring Manager,

I came across your job posting for a Data Science Intern and was impressed by the exciting projects your team is working on. As a Business Development Executive at AtliQ, I'd like to introduce you to our company's capabilities in delivering top-notch data science solutions that can support your business goals.

At AtliQ, we specialize in harnessing the power of AI and software consulting to drive business growth, scalability, and efficiency. Our team of experts has a proven track record of developing and implementing predictive models, natural language processing, and generative AI algorithms that drive actionable insights and informed decision-making.

Our expertise in machine learning, deep learning, and Python programming aligns perfectly with your requirements. We've successfully delivered projects that involve feature engineering, model development, and application of AI technologies to drive bus

In [22]:
job

[{'role': 'Data Science Intern (m/f/d)',
  'experience': 'Intern for at least six months',
  'skills': ['ML',
   'Deep learning',
   'NLP',
   'Generative AI algorithms',
   'Python',
   'SQL',
   'Statistics'],
  'descriptions': ['Assist in the analysis and interpretation of data to derive actionable insights',
   'Contribute to the feature engineering process that improve the accuracy and performance of our models',
   'Support the development and application of predictive and forecasting models',
   'Contribute to projects involving natural language processing',
   'Research the application of the newest AI technologies']}]